Import required libraries

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker
sns.set(style="ticks")

Import data for input visualization

In [ ]:
Data_OID = pd.read_csv('Data_OID_Loops_CSB_Reduced.csv', delimiter=",")


In [ ]:
SubData_OID_50 = Data_OID.query('hour == 50')
SubData_OID_50_type = SubData_OID_50.query('type == "PARAM_FIT"')
SubData_OID_50_type.trueValue.unique()


In [ ]:
d = SubData_OID_50_type.set_index('runName') # set the name of the run as index to the dataframe
d['abslog2Ratio'] = np.absolute(np.log2(d['value']/d['trueValue']))   # add a column to the dataframe with the absolute value of the log2-ratio between estimate and true parameter value, for each experiment
d['log2Ratio'] = np.log2(d['value']/d['trueValue'])   # add a column to the dataframe with the absolute value of the log2-ratio between estimate and true parameter value, for each experiment

SubData_OID_50_type_update = d.reset_index()



In [ ]:
SubData_OID_50_type_update.loc[SubData_OID_50_type_update['numLoops']==1,'LoopsLabel'] = 'O1'
SubData_OID_50_type_update.loc[SubData_OID_50_type_update['numLoops']==3,'LoopsLabel'] = 'O3'
SubData_OID_50_type_update.loc[SubData_OID_50_type_update['numLoops']==5,'LoopsLabel'] = 'O5'


Now extracting the dataframe for the convergence

In [ ]:
SubData_OID_type = Data_OID.query('type == "PARAM_FIT"')
d = SubData_OID_type.set_index('runName') # set the name of the run as index to the dataframe
d['abslog2Ratio'] = np.absolute(np.log2(d['value']/d['trueValue']))   # add a column to the dataframe with the absolute value of the log2-ratio between estimate and true parameter value, for each experiment
d['log2Ratio'] = np.log2(d['value']/d['trueValue'])   # add a column to the dataframe with the absolute value of the log2-ratio between estimate and true parameter value, for each experiment

SubData_OID_type_update = d.reset_index()
SubData_OID_type_update.loc[SubData_OID_type_update['numLoops']==1,'LoopsLabel'] = 'O1'
SubData_OID_type_update.loc[SubData_OID_type_update['numLoops']==3,'LoopsLabel'] = 'O3'
SubData_OID_type_update.loc[SubData_OID_type_update['numLoops']==5,'LoopsLabel'] = 'O5'


In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(6.8, 5.8), dpi = 600)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
pal = dict(alpha1='#fff7f3', Vm1 = '#fde0dd',h1 = '#fcc5c0', Km1 = '#fa9fb5', d1 ='#f768a1' , alpha2 = '#dd3497', d2 ='#ae017e' ,Kf='#7a0177' )

gs0 = gridspec.GridSpec(2, 1)
gs0.update(left=0.05, right=0.35)
ax0 = plt.subplot(gs0[0, :])


#------> Bar plot of average relative error 
pal1 = dict(O1='#fcbba1', O3 = '#fb6a4a',O5 = '#cb181d')
import matplotlib.patches as patches
ax0 = plt.subplot(gs0[0, :])
#ax0 = plt.axhline(y=0,c = 'black',linewidth = 0.5)
ax0.add_patch(patches.Rectangle((-0.5,0),1,0.5,alpha = .05))
ax0 = sns.barplot(data = SubData_OID_50_type_update,x = "LoopsLabel",y = "abslog2Ratio",order =['O1','O3','O5'], palette = pal1,edgecolor = 'black',errwidth = 0.8,alpha=.8)
ax0.spines['top'].set_visible(False)
ax0.spines['right'].set_visible(False)
ax0.tick_params(length=2)
ax0.set_xticklabels(('1','3','5'),rotation = 0)
ax0.set_xlabel('OID Loops')
ax0.set_ylabel(r'$\bar{\varepsilon}$ (-)',rotation = 0)

#------> Convergence rate of the average error
ax2 = plt.subplot(gs0[1, :])
ax2 = sns.pointplot(x='hour', y='abslog2Ratio',hue= 'LoopsLabel', data=SubData_OID_type_update,sharey=False,marker = '.',scale = 0.7,errwidth=2,palette=['#fcbba1','#fb6a4a','#cb181d'],alpha=.8)#,line_kws=line_kws)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.tick_params(length=2)
ax2.set_xlabel('Time (hours)')
ax2.set_ylabel(r'$\bar{\varepsilon}$ (-)',rotation = 0)


#------> Box+Swarm plot of relative error for each input class and parameter
gs1 = gridspec.GridSpec(3, 1)
gs1.update(left=0.55, hspace=0.1)#0.05

ax3 = plt.subplot(gs1[0, :])
ax3.add_patch(patches.Rectangle((-0.5,-0.5),8.5,4.5,alpha = .05))
ax3 = plt.axhline(y=0,c='b',linewidth=1)
ax3 = sns.swarmplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O1",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O1",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax3 = sns.boxplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O1",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O1",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.tick_params(length=2)
ax3.set_xticklabels('')
ax3.set_ylabel('')
ax3.set_ylim([-0.5,4])
ax3.set_title('1 Loop',y = 0.85)

ax4 = plt.subplot(gs1[1, :])
ax4 = plt.axhline(y=0,c='b',linewidth=1)
ax4 = sns.swarmplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O3",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O3",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax4 = sns.boxplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O3",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O3",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax4.tick_params(length=2)
ax4.set_xticklabels('')
ax4.set_ylabel('')
ax4.set_title('3 Loops',y = 0.85)
ax4.set_ylim([-0.5,4])

ax4.text(-2.5, -0.5, r'$\varepsilon$(-)')


ax5 = plt.subplot(gs1[2, :])
ax5 = plt.axhline(y=0,c='b',linewidth=1)
ax5 = sns.swarmplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O5",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O5",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax5 = sns.boxplot(SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O5",'param'],SubData_OID_50_type_update.loc[SubData_OID_50_type_update['LoopsLabel'] == "O5",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax5.spines['right'].set_visible(False)
ax5.spines['top'].set_visible(False)
ax5.tick_params(length=2)
ax5.set_xticklabels([r'$\alpha$',r'$v$',r'$h$',r'$K_{r}$',r'$\gamma$',r'$k_{p}$',r'$\gamma_{f}$',r'$k_{f}$'])
ax5.set_ylabel('')
ax5.set_xlabel('Parameter')
ax5.set_ylim([-0.5,4])
#ax5.set_yscale('log')
ax5.set_title('5 Loops',y = 0.85)

for ax in [ax3, ax4]:
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.set_xlabel('')



plt.show()
